In [ ]:
import json
import os
import shutil
import pandas as pd
from pathlib import Path
from time import sleep
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

dirpath = os.getcwd()+'\\chromedriver.exe' 
download_path = os.getcwd()+'\\output'
chrome_options = Options()
chrome_options.add_argument('--start-maximized')
chrome_options.add_argument("--enable-javascript")

appState = {
   "recentDestinations": [
        {
            "id": "Save as PDF",
            "origin": "local",
            "account": ""
        }
    ],
    "selectedDestinationId": "Save as PDF",
    "version": 2
}
prefs = {
    'printing.print_preview_sticky_settings.appState': json.dumps(appState),
    'savefile.default_directory': download_path
        }

chrome_options.add_experimental_option('prefs', prefs)
chrome_options.add_argument('--kiosk-printing')

browser = webdriver.Chrome(executable_path=dirpath, options=chrome_options) #instancia o driver

dic = {"Produto": [],
       "Preço": [],
       "Desconto %": [],
       "Quantidade": []
}

with open('links.json', encoding='utf-8') as fh:
    json = json.load(fh)

In [ ]:
for x in json['Grupo BIG']:
    browser.get("{}".format(json['Grupo BIG'][x]))
    itens = browser.find_elements_by_xpath("*//h2[contains(text(), '{}' )]//..//..//div[2]//ul//div".format(x))

In [ ]:
itens[0].text

In [ ]:
porcentagem = []
for x in itens:
    if re.search("%", x.text) and len(x.text.split("\n")) >=4:
        porcentagem.append(x)

        
for x in porcentagem:
    list_temp = x.text.split("\n")
    if len(list_temp) == 6:
        list_temp.remove(list_temp[2])
        list_temp.remove(list_temp[0])
    else:
        list_temp.remove(list_temp[1])
        
    dic["Produto"].append(list_temp[2])
    dic["Preço"].append(list_temp[0])
    dic["Desconto %"].append(list_temp[1])
    dic["Quantidade"].append(list_temp[3])
    
df = pd.DataFrame(dic).drop_duplicates(ignore_index=True)
df['Desconto %'] = df['Desconto %'].str.replace("%", '')
df['Desconto %'] = df['Desconto %'].astype('float64')
df = df.sort_values('Desconto %', ascending=False, ignore_index=True)

In [ ]:
df